In [2]:
from Bio import Entrez, SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import numpy as np
import pandas as pd
import re

import AS_motif_analysis as AS

In [4]:
# input significant alternative splicing (Skipped exons) evants with splicing sites coodinate
a5ss_sig = pd.read_csv('/home/yu/R_studio_work_dir/AS_further_analysis/output/A5SS_sig.csv')
a5ss_sig.iloc[:7,]

,ID,GeneID,geneSymbol,chr,strand,longExonStart_0base,longExonEnd,shortES,shortEE,flankingES,...,IncLevelDifference,IncLevel1-1,IncLevel1-2,IncLevel1-3,IncLevel1-4,IncLevel2-1,IncLevel2-2,IncLevel2-3,IncLevel2-4,AS_type
0,1300,ENSG00000177697,CD151,chr11,+,832993,833026,832993,833022,834529,...,0.090,1.000,1.000,1.000,1.0,0.868,1.000,0.935,0.836,A5SS
1,3715,ENSG00000170919,TPT1-AS1,chr13,+,45379194,45380001,45379194,45379335,45383090,...,-0.633,0.600,0.067,0.600,0.2,1.000,1.000,1.000,1.000,A5SS
2,1328,ENSG00000105223,PLD3,chr19,+,40365552,40365930,40365552,40365585,40366418,...,0.111,0.881,1.000,1.000,1.0,0.828,0.891,0.919,0.799,A5SS
3,5173,ENSG00000206149,HERC2P9,chr15,+,28641169,28641312,28641169,28641226,28641429,...,0.492,1.000,1.000,1.000,1.0,0.390,0.793,0.390,0.460,A5SS
4,1838,ENSG00000188243,COMMD6,chr13,-,75537498,75537675,75537663,75537675,75530113,...,0.490,1.000,1.000,1.000,1.0,0.009,1.000,1.000,0.030,A5SS
5,1458,ENSG00000183401,CCDC159,chr19,+,11346504,11346634,11346504,11346627,11349653,...,-0.524,0.643,0.286,0.536,0.0,0.562,1.000,1.000,1.000,A5SS
6,4115,ENSG00000172269,DPAGT1,chr11,-,119101017,119102142,119101494,119102142,119100261,...,0.537,0.333,1.000,1.000,1.0,0.273,0.200,0.111,0.600,A5SS


In [8]:
## Calculate the length of upstream,dowstream exon and intron, and flanked exon, 
## and calcualte the genome coodinates in order to download from NCBI
a5ss_sig_as_coord_len = pd.DataFrame(columns = ['ID','GeneID','geneSymbol','chr','strand',
                                                'seq_1_in_co1','seq_1_in_co2','seq_1_in_co3','seq_1_in_co4',
                                                'seq_1_ex_co1','seq_1_ex_co2','seq_1_ex_co3','seq_1_ex_co4',
                                                'seq_2_in_co1','seq_2_in_co2','seq_2_in_co3','seq_2_in_co4',
                                                'seq_2_ex_co1','seq_2_ex_co2','seq_2_ex_co3','seq_2_ex_co4'])
for i in range(0,len(a5ss_sig)):
    if a5ss_sig['strand'][i] == '+':
        a5ss_sig_as_coord_len.loc[i,'seq_1_ex_co2'] = a5ss_sig.loc[i,'shortEE'].copy()
        a5ss_sig_as_coord_len.loc[i,'seq_1_ex_co3'] = a5ss_sig.loc[i,'shortEE'].copy() + 1
        a5ss_sig_as_coord_len.loc[i,'seq_1_in_co2'] = a5ss_sig.loc[i,'longExonEnd'].copy()
        a5ss_sig_as_coord_len.loc[i,'seq_1_in_co3'] = a5ss_sig.loc[i,'longExonEnd'].copy() + 1
        a5ss_sig_as_coord_len.loc[i,'seq_2_ex_co2'] = a5ss_sig.loc[i,'flankingES'].copy() - 1
        a5ss_sig_as_coord_len.loc[i,'seq_2_ex_co3'] = a5ss_sig.loc[i,'flankingES'].copy()
        a5ss_sig_as_coord_len.loc[i,'seq_2_in_co2'] = a5ss_sig.loc[i,'flankingES'].copy() - 1
        a5ss_sig_as_coord_len.loc[i,'seq_2_in_co3'] = a5ss_sig.loc[i,'flankingES'].copy()
        
        
        if (a5ss_sig.loc[i,'shortEE'] - a5ss_sig.loc[i,'shortES'] + 1) >= 150:
            a5ss_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = a5ss_sig.loc[i,'shortEE'].copy() - 150 + 1
        else:
            a5ss_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = a5ss_sig.loc[i,'shortES'].copy()
        
        if (a5ss_sig.loc[i,'longExonEnd'] - a5ss_sig.loc[i,'longExonStart_0base'] + 1) >= 150:
            a5ss_sig_as_coord_len.loc[i,'seq_1_in_co1'] = a5ss_sig.loc[i,'longExonEnd'].copy() - 150 + 1
        else:
            a5ss_sig_as_coord_len.loc[i,'seq_1_in_co1'] = a5ss_sig.loc[i,'longExonStart_0base'].copy()
        
        if (a5ss_sig.loc[i,'flankingES'] - a5ss_sig.loc[i,'shortEE'] - 1) >= 300:
            a5ss_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = a5ss_sig.loc[i,'shortEE'].copy() + 150
            a5ss_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = a5ss_sig.loc[i,'flankingES'].copy() - 150
        else:
            a5ss_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = a5ss_sig.loc[i,'shortEE'].copy() + int((a5ss_sig.loc[i,'flankingES'] - a5ss_sig.loc[i,'shortEE'] - 1)/2)
            a5ss_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = a5ss_sig.loc[i,'flankingES'].copy() - int((a5ss_sig.loc[i,'flankingES'] - a5ss_sig.loc[i,'shortEE'] - 1)/2)
            
        if (a5ss_sig.loc[i,'flankingES'] - a5ss_sig.loc[i,'longExonEnd'] - 1) >= 300:
            a5ss_sig_as_coord_len.loc[i,'seq_1_in_co4'] = a5ss_sig.loc[i,'longExonEnd'].copy() + 150
            a5ss_sig_as_coord_len.loc[i,'seq_2_in_co1'] = a5ss_sig.loc[i,'flankingES'].copy() - 150
        else:
            a5ss_sig_as_coord_len.loc[i,'seq_1_in_co4'] = a5ss_sig.loc[i,'longExonEnd'].copy() + int((a5ss_sig.loc[i,'flankingES'] - a5ss_sig.loc[i,'longExonEnd'] - 1)/2)
            a5ss_sig_as_coord_len.loc[i,'seq_2_in_co1'] = a5ss_sig.loc[i,'flankingES'].copy() - int((a5ss_sig.loc[i,'flankingES'] - a5ss_sig.loc[i,'longExonEnd'] - 1)/2)

        if (a5ss_sig.loc[i,'flankingEE'] - a5ss_sig.loc[i,'flankingES'] + 1) >= 150:
            a5ss_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = a5ss_sig.loc[i,'flankingES'].copy() + 150 - 1
            a5ss_sig_as_coord_len.loc[i,'seq_2_in_co4'] = a5ss_sig.loc[i,'flankingES'].copy() + 150 - 1
        else:
            a5ss_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = a5ss_sig.loc[i,'flankingEE'].copy()
            a5ss_sig_as_coord_len.loc[i,'seq_2_in_co4'] = a5ss_sig.loc[i,'flankingEE'].copy()
            
    else:
        a5ss_sig_as_coord_len.loc[i,'seq_1_ex_co2'] = a5ss_sig.loc[i,'shortES'].copy()
        a5ss_sig_as_coord_len.loc[i,'seq_1_ex_co3'] = a5ss_sig.loc[i,'shortES'].copy() - 1
        a5ss_sig_as_coord_len.loc[i,'seq_1_in_co2'] = a5ss_sig.loc[i,'longExonStart_0base'].copy()
        a5ss_sig_as_coord_len.loc[i,'seq_1_in_co3'] = a5ss_sig.loc[i,'longExonStart_0base'].copy() - 1
        a5ss_sig_as_coord_len.loc[i,'seq_2_ex_co2'] = a5ss_sig.loc[i,'flankingEE'].copy() + 1
        a5ss_sig_as_coord_len.loc[i,'seq_2_ex_co3'] = a5ss_sig.loc[i,'flankingEE'].copy()
        a5ss_sig_as_coord_len.loc[i,'seq_2_in_co2'] = a5ss_sig.loc[i,'flankingEE'].copy() + 1
        a5ss_sig_as_coord_len.loc[i,'seq_2_in_co3'] = a5ss_sig.loc[i,'flankingEE'].copy()
        
        
        if (a5ss_sig.loc[i,'shortEE'] - a5ss_sig.loc[i,'shortES'] + 1) >= 150:
            a5ss_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = a5ss_sig.loc[i,'shortES'].copy() + 150 - 1
        else:
            a5ss_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = a5ss_sig.loc[i,'shortEE'].copy()
        
        if (a5ss_sig.loc[i,'longExonEnd'] - a5ss_sig.loc[i,'longExonStart_0base'] + 1) >= 150:
            a5ss_sig_as_coord_len.loc[i,'seq_1_in_co1'] = a5ss_sig.loc[i,'longExonStart_0base'].copy() + 150 - 1
        else:
            a5ss_sig_as_coord_len.loc[i,'seq_1_in_co1'] = a5ss_sig.loc[i,'longExonEnd'].copy()
            
        
        if (a5ss_sig.loc[i,'shortES'] - a5ss_sig.loc[i,'flankingEE'] - 1) >= 300:
            a5ss_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = a5ss_sig.loc[i,'shortES'].copy() - 150
            a5ss_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = a5ss_sig.loc[i,'flankingEE'].copy() + 150
        else:
            a5ss_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = a5ss_sig.loc[i,'shortES'].copy() - int((a5ss_sig.loc[i,'shortES'] - a5ss_sig.loc[i,'flankingEE'] - 1)/2)
            a5ss_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = a5ss_sig.loc[i,'flankingEE'].copy() + int((a5ss_sig.loc[i,'shortES'] - a5ss_sig.loc[i,'flankingEE'] - 1)/2)
            
        if (a5ss_sig.loc[i,'longExonStart_0base'] - a5ss_sig.loc[i,'flankingEE'] - 1) >= 300:
            a5ss_sig_as_coord_len.loc[i,'seq_1_in_co4'] = a5ss_sig.loc[i,'longExonStart_0base'].copy() - 150
            a5ss_sig_as_coord_len.loc[i,'seq_2_in_co1'] = a5ss_sig.loc[i,'flankingEE'].copy() + 150
        else:
            a5ss_sig_as_coord_len.loc[i,'seq_1_in_co4'] = a5ss_sig.loc[i,'longExonStart_0base'].copy() - int((a5ss_sig.loc[i,'longExonStart_0base'] - a5ss_sig.loc[i,'flankingEE'] - 1)/2)
            a5ss_sig_as_coord_len.loc[i,'seq_2_in_co1'] = a5ss_sig.loc[i,'flankingEE'].copy() + int((a5ss_sig.loc[i,'longExonStart_0base'] - a5ss_sig.loc[i,'flankingEE'] - 1)/2)

        if (a5ss_sig.loc[i,'flankingEE'] - a5ss_sig.loc[i,'flankingES'] + 1) >= 150:
            a5ss_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = a5ss_sig.loc[i,'flankingEE'].copy() - 150 + 1
            a5ss_sig_as_coord_len.loc[i,'seq_2_in_co4'] = a5ss_sig.loc[i,'flankingEE'].copy() - 150 + 1
        else:
            a5ss_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = a5ss_sig.loc[i,'flankingES'].copy()
            a5ss_sig_as_coord_len.loc[i,'seq_2_in_co4'] = a5ss_sig.loc[i,'flankingES'].copy()
            


a5ss_sig_as_coord_len.iloc[:,:5] = a5ss_sig.iloc[:,:5].copy()
a5ss_sig_as_coord_len

,ID,GeneID,geneSymbol,chr,strand,seq_1_in_co1,seq_1_in_co2,seq_1_in_co3,seq_1_in_co4,seq_1_ex_co1,...,seq_1_ex_co3,seq_1_ex_co4,seq_2_in_co1,seq_2_in_co2,seq_2_in_co3,seq_2_in_co4,seq_2_ex_co1,seq_2_ex_co2,seq_2_ex_co3,seq_2_ex_co4
0,1300,ENSG00000177697,CD151,chr11,+,832993,833026,833027,833176,832993,...,833023,833172,834379,834528,834529,834678,834379,834528,834529,834678
1,3715,ENSG00000170919,TPT1-AS1,chr13,+,45379852,45380001,45380002,45380151,45379194,...,45379336,45379485,45382940,45383089,45383090,45383235,45382940,45383089,45383090,45383235
2,1328,ENSG00000105223,PLD3,chr19,+,40365781,40365930,40365931,40366080,40365552,...,40365586,40365735,40366268,40366417,40366418,40366510,40366268,40366417,40366418,40366510
3,5173,ENSG00000206149,HERC2P9,chr15,+,28641169,28641312,28641313,28641370,28641169,...,28641227,28641327,28641371,28641428,28641429,28641578,28641328,28641428,28641429,28641578
4,1838,ENSG00000188243,COMMD6,chr13,-,75537647,75537498,75537497,75537348,75537675,...,75537662,75537513,75530416,75530267,75530266,75530117,75530416,75530267,75530266,75530117
5,1458,ENSG00000183401,CCDC159,chr19,+,11346504,11346634,11346635,11346784,11346504,...,11346628,11346777,11349503,11349652,11349653,11349687,11349503,11349652,11349653,11349687
6,4115,ENSG00000172269,DPAGT1,chr11,-,119101166,119101017,119101016,119100867,119101643,...,119101493,119101344,119100558,119100409,119100408,119100408,119100558,119100409,119100408,119100408
7,4058,ENSG00000115677,HDLBP,chr2,-,241269102,241268953,241268952,241268803,241269295,...,241269199,241269050,241268691,241268542,241268541,241268541,241268691,241268542,241268541,241268541
8,4060,ENSG00000115677,HDLBP,chr2,-,241269106,241268957,241268956,241268807,241269295,...,241269199,241269050,241268691,241268542,241268541,241268541,241268691,241268542,241268541,241268541
9,4974,ENSG00000158092,NCK1,chr3,+,136862207,136862353,136862354,136862503,136862207,...,136862320,136862469,136927833,136927982,136927983,136928132,136927833,136927982,136927983,136928132
